In [ ]:
%load_ext pycodestyle_magic

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import json
import tempfile

result = []
result.append(json.dumps({}))
# SETUP
def cluster(wineData_scaled, n_clusters):
    model = KMeans(n_clusters=n_clusters, n_init="auto", random_state=2023)
    model.fit(wineData_scaled)
    return model

wineData_raw = pd.read_csv(f"./wineQualityReds.csv")

In [8]:
def preprocess(wineDataFrame):
    # 1. drop "Wine" column which just assigns a number (1, 2, ...) to each wine
    # It is okay to modify wineDataFrame
    wineDataFrame.drop("Wine", axis=1, inplace=True)

    # 2. split into data and labels. It is okay to modify wineDataFrame.
    quality = wineDataFrame["quality"]
    wineDataFrame.drop("quality", axis=1, inplace=True)

    # 3. scale data but don't overwrite raw data
    # 4. return data and labels
    scaler = StandardScaler()
    scaler.fit(wineDataFrame)
    return pd.DataFrame(scaler.transform(wineDataFrame), columns=wineDataFrame.columns), quality

wineData_scaled, quality = preprocess(wineData_raw)
k = 6
model = cluster(wineData_scaled, k)

In [9]:
# VALIDATION
wineData_scaled["Cluster"] = pd.Series(model.predict(wineData_scaled))
wineData_scaled['quality'] = quality
quality_means = []
Ns = []
for i in range(k):
    cluster_df = wineData_scaled[wineData_scaled["Cluster"] == i]
    quality_means.append(round(cluster_df["quality"].mean(), 3))
    Ns.append(cluster_df.shape[0])
quality_df = pd.DataFrame({"quality": quality_means, "N": Ns})
quality_df.sort_values("quality", ascending=True, inplace=True)
result.append(json.dumps(quality_df.to_csv()))

   quality    N
5    5.318  513
2    5.322  339
3    5.357   28
1    5.864  287
0    5.877  195
4    6.333  237
